In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
from pathlib import Path
from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix,classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

EMOTIONS=['anger','contempt','disgust','fear',
          'happy','neutral','sad','surprise']

NUM_CLASSES=8
BATCH_SIZE=32
EPOCHS=25
IMG_SIZE=224


In [5]:
class AffectNetDataset(Dataset):

    def __init__(self,root,split,transform=None):
        self.transform=transform
        self.images=[]
        self.labels=[]

        root=Path(root)/split

        for i,e in enumerate(EMOTIONS):
            p=root/e
            if p.exists():
                for img in p.glob("*.*"):
                    self.images.append(img)
                    self.labels.append(i)

    def __len__(self):
        return len(self.images)

    def __getitem__(self,idx):
        img=Image.open(self.images[idx]).convert("RGB")
        if self.transform:
            img=self.transform(img)
        return img,self.labels[idx]


In [6]:
train_tf=transforms.Compose([
 transforms.Resize((IMG_SIZE,IMG_SIZE)),
 transforms.RandomHorizontalFlip(),
 transforms.RandomRotation(15),
 transforms.ColorJitter(0.3,0.3,0.3,0.1),
 transforms.RandomResizedCrop(224,scale=(0.8,1)),
 transforms.ToTensor(),
 transforms.Normalize([0.485,0.456,0.406],
                      [0.229,0.224,0.225])
])

test_tf=transforms.Compose([
 transforms.Resize((IMG_SIZE,IMG_SIZE)),
 transforms.ToTensor(),
 transforms.Normalize([0.485,0.456,0.406],
                      [0.229,0.224,0.225])
])
